In [1]:
# Opłata na Start - 10%
# Okres Umowy (miesiące) - 36
# Roczny limit km - 20 tys.
# Leasing

# Imports

In [2]:
import io
import pickle
import time

import numpy as np
import pandas as pd
import requests
from tqdm import tqdm 

from bs4 import BeautifulSoup  # for processing html structure of website

import selenium  # Python Selenium
from selenium import webdriver  # for specifying webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.support import expected_conditions as EC  # for checking visibility of an element
from selenium.webdriver.support.ui import WebDriverWait  # this three enable waiting until sth is displayed on website
from selenium.webdriver.common.by import By  # for checking element visibility by XPath
from webdriver_manager.chrome import ChromeDriverManager  # chromedriver for automatized access to Chrome

def save_object(obj, filename):  # function defined for saving Python objects
    with open(filename, 'wb') as output:  # overwrites any existing file
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# Request URL

In [12]:
# url = "https://automarket.pl/oferta/dacia/spring/209864/leasing?f=IjM2LDEwLDIwMDAwLDIi&is_company=0#images"
url = "https://automarket.pl/oferta/opel/corsa/207347/leasing?f=IjM2LDEwLDIwMDAwLDIi"
url = "https://automarket.pl/oferta/citroen/c3/144362/leasing?f=IjM2LDEwLDIwMDAwLDIi&is_company=0"

In [13]:
webpage_html = requests.get(url) # request for a webpage structure

In [14]:
soup = BeautifulSoup(webpage_html.text, 'html.parser') # one need to format it into a BeautifulSoup object before proceeding

In [15]:
webpage_html.text

'<!DOCTYPE html><html  lang="pl" data-capo=""><head><meta charset="utf-8">\n<meta name="viewport" content="width=device-width, user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0">\n<title>Używany CITROEN C3 w leasingu na Automarket.pl | Swarzędz</title>\n<link rel="preconnect" href="https://fonts.googleapis.com" crossorigin="anonymous">\n<link rel="preconnect" href="https://cdn.jsdelivr.net" crossorigin="anonymous">\n<style>#html-body [data-pb-style=D6KQMVN]{background-color:#f3fdfb;background-position:left top;background-size:cover;background-repeat:no-repeat;background-attachment:scroll;padding-left:16px;padding-top:84px;padding-bottom:139px}#html-body [data-pb-style=HDQ8W15]{display:flex;flex-direction:column}#html-body [data-pb-style=SS0PGJ0]{text-align:center;margin-bottom:17px}#html-body [data-pb-style=Q2C0G8I]{margin-bottom:28px}#html-body [data-pb-style=EU2P95M]{background-position:left top;background-size:cover;background-repeat:no-repeat;background-atta

## Get Numer Oferty, Price, Location

In [16]:
try:
    price = soup.find('p', class_='text-2xl lg:text-[32px] leading-8 font-semibold text-pko-primary') or soup.find('p', class_='text-2xl lg:text-[32px] leading-8 font-semibold !text-pko-red-500 text-pko-primary')
    if price:
        price = price.text.replace('\xa0', ' ')
    else:
        price = None  # Handle the case where neither class is found
except Exception as e:
    price = None  # Handle any exceptions that occur
    
numer_oferty = soup.find('p', class_='text-base text-pko-secondary mt-5 text-center mb-8 md:ml-5 md:my-0 leading-4').text.split(': ')[1]
lokalizacja_samochodu = soup.find('p', class_='text-sm leading-sm py-3 md:py-0 text-pko-grey-1000 leading-[14px]').find('span').text.strip()

In [ ]:
print(f'''Numer oferty: {numer_oferty}
Cena: {price}
Lokalizacja: {lokalizacja_samochodu}''')

## Get Car Model

In [6]:
whole_model = soup.find('h1')
car_model = whole_model.find_all('span')[0].text.strip()
car_model_type = whole_model.find_all('span')[1].text.strip()

In [ ]:
print("Car Model:", car_model)
print("Car Model Type:", car_model_type)

## Get Informacje Podstawowe

In [8]:
informacje_podstawowe = soup.find('h2').find_next().find_all('span')
rok_produkcji = informacje_podstawowe[1].text
przebieg = informacje_podstawowe[3].text
naped = informacje_podstawowe[5].text
skrzynia_biegow = informacje_podstawowe[7].text

In [ ]:
print(f'''Rok produkcji: {rok_produkcji}
Przebieg: {przebieg}
Naped: {naped}
Skrzynia biegow: {skrzynia_biegow}''')

In [10]:
attributes = ["Pojemność silnika", "Moc silnika", "Kolor", "Typ nadwozia", "Liczba miejsc", "Kraj pochodzenia"]
pojazd_info = {attr: soup.find(string=lambda text: text and attr in text).find_next().text for attr in attributes}

pojemnosc_silnika = pojazd_info["Pojemność silnika"]
moc_silnika = pojazd_info["Moc silnika"]
kolor = pojazd_info["Kolor"]
typ_nadwozia = pojazd_info["Typ nadwozia"]
liczba_miejsc = pojazd_info["Liczba miejsc"]
kraj_pochodzenia = pojazd_info["Kraj pochodzenia"]

In [ ]:
print(f'''pojemność silnika: {pojemnosc_silnika}
moc silnika: {moc_silnika}
kolor: {kolor}
typ nadwozia: {typ_nadwozia}
liczba miejsc: {liczba_miejsc}
kraj pochodzenia: {kraj_pochodzenia}''')

## Get Wymiary

In [12]:
dimensions = ["Długość", "Szerokość", "Wysokość", "Rozstaw osi", "Masa całkowita"]
dimension_values = {dim: soup.find(string=lambda text: text and dim in text).find_next().text for dim in dimensions}

dlugosc = dimension_values["Długość"]
szerokosc = dimension_values["Szerokość"]
wysokosc = dimension_values["Wysokość"]
rozstaw_osi = dimension_values["Rozstaw osi"]
masa_calkowita = dimension_values["Masa całkowita"]

In [ ]:
print(f'''długość: {dlugosc}
szerokość: {szerokosc}
wysokość: {wysokosc}
rozstaw osi: {rozstaw_osi}
masa całkowita: {masa_calkowita}''')